In [1]:
import augmenty 

/home/coder/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/coder/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Testing Spacy Models & Augmenty

In [42]:
import spacy
#load spacy model
nlp = spacy.load("da_core_news_md")

#define augmenter dictionary
names = {"first_name": ["Laura", "Peter", "Poul"],
         "last_name": ["Petersen", "Jensen", "Andersen"]}

patterns = [["first_name"], ["first_name", "last_name"],
            ["first_name", "last_name", "last_name"]]

person_tag = "PER"

# replace 100% of names:
per_augmenter = augmenty.load("per_replace.v1", patterns = patterns, names = names, level = 1, person_tag = "PER", replace_consistency = True)

# text
texts = ["Anton Antonsen er rigtig god til at tale spansk", "Lars Jensen elsker at danse salsa", "Lars Krogh elsker pizza"]

# augment text with custom augmenter
list(augmenty.texts(texts, per_augmenter, nlp))

['Poul Petersen er rigtig god til at tale spansk',
 'Laura Andersen elsker at danse salsa',
 'Laura Jensen Jensen elsker pizza']

In [43]:
doc = nlp("Anton Antonsen er rigtig god til at tale spansk")

for ent in doc.ents:
   print(ent, ent.label_)

Anton Antonsen PER
spansk MISC


## Testing Flair and Polyglot

In [19]:
from danlp.models import load_flair_ner_model
from flair.data import Sentence

#import flair ner model 
tagger = load_flair_ner_model()

#text 
text = "Morten Østergaard bor i København tæt på Kongens Nytorv"
sentence = Sentence(text)

tagger.predict(sentence)

2023-01-29 12:57:39,634 loading file /home/coder/.danlp/flair.ner.pt


In [39]:
for entity in sentence.get_spans('ner'):
    print(entity)

Span [1,2]: "Morten Østergaard"   [− Labels: PER (0.9968)]
Span [5]: "København"   [− Labels: LOC (0.9944)]
Span [8,9]: "Kongens Nytorv"   [− Labels: LOC (0.6427)]


In [ ]:
import polyglot
from polyglot.text import Text, Word

text = Text(u"Peter Jensen elsker at danse salsa", hint_language_code="da")
print(text.entities)

## Dacy Models

In [47]:
dacy.models()

['da_dacy_small_tft-0.0.0',
 'da_dacy_medium_tft-0.0.0',
 'da_dacy_large_tft-0.0.0',
 'da_dacy_small_trf-0.1.0',
 'da_dacy_medium_trf-0.1.0',
 'da_dacy_large_trf-0.1.0']

In [2]:
import dacy 
nlp = dacy.load("da_dacy_large_trf-0.1.0")

In [3]:
doc = nlp("Anton Antonsen er rigtig god til at tale spansk")

for ent in doc.ents:
   print(ent, ent.label_)

Anton Antonsen PER
spansk MISC


/home/coder/.local/lib/python3.9/site-packages/spacy/pipeline/attributeruler.py:150: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)


## STANZA

In [1]:
import stanza
from tqdm import tqdm

nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
doc = nlp("Anton Antonsen er rigtig god til at tale spansk")
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep='\n')

2023-01-29 14:22:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-29 14:22:45 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2023-01-29 14:22:45 INFO: Use device: cpu
2023-01-29 14:22:45 INFO: Loading: tokenize
2023-01-29 14:22:45 INFO: Loading: ner
2023-01-29 14:22:47 INFO: Done loading processors!


token: Anton	ner: B-PERSON
token: Antonsen	ner: E-PERSON
token: er	ner: O
token: rigtig	ner: O
token: god	ner: O
token: til	ner: O
token: at	ner: O
token: tale	ner: O
token: spansk	ner: O


## Testing with DACY datasets

In [8]:
from dacy.datasets import dane, danish_names, female_names, male_names, muslim_names, load_names

m_name_dict = male_names()

/home/coder/.local/lib/python3.9/site-packages/dacy/datasets/names.py:50: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  names = names.groupby(level=0).apply(lambda x: x / float(x.sum()))


In [43]:
import augmenty
import spacy 
nlp = spacy.load("da_core_news_sm")
patterns = [["first_name"], ["first_name", "last_name"], ["first_name", "last_name", "last_name"]]

# replace 100% of names:
per_augmenter = augmenty.load("per_replace.v1", patterns = patterns, names = m_name_dict, level = 1, person_tag = "PER", replace_consistency = True)

# text
texts = ["Anton Antonsen er rigtig god til at tale spansk", "Lars Jensen elsker at danse salsa", "Lars Krogh elsker pizza"]

# augment text with custom augmenter
list(augmenty.texts(texts, per_augmenter, nlp))

/home/coder/.local/lib/python3.9/site-packages/spacy/util.py:762: UserWarning: [W095] Model 'da_core_news_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['Raymond Midtgaard Hvid er rigtig god til at tale spansk',
 'Peer Hamann elsker at danse salsa',
 'Regnar Baun Laustsen elsker pizza']